# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,23.91,94,100,2.06,US,1684557809
1,1,bethel,41.3712,-73.4140,12.74,87,0,2.06,US,1684557810
2,2,kodiak,57.7900,-152.4072,6.73,93,100,6.69,US,1684557810
3,3,edinburgh of the seven seas,-37.0676,-12.3116,17.22,97,96,8.21,SH,1684557810
4,4,tadine,-21.5500,167.8833,23.14,55,23,5.06,NC,1684557810


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [64]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [60]:
# Narrow down cities that fit criteria and drop any results with null values
city_filtered = city_data_df.loc[(city_data_df["Cloudiness"]==0)&
                                 (city_data_df["Max Temp"]>=18)&(city_data_df["Max Temp"]<=22)&
                                (city_data_df["Wind Speed"]<=4)]

# Drop any rows with null values
city_filtered.dropna(how="any")

# Display sample data
city_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,namibe,-15.1961,12.1522,18.51,70,0,2.54,AO,1684557818
86,86,margaret river,-33.9500,115.0667,20.05,32,0,1.12,AU,1684557822
180,180,bom jesus da lapa,-13.2550,-43.4181,20.25,61,0,1.17,BR,1684557834
190,190,urangan,-25.3000,152.9000,20.01,47,0,3.92,AU,1684557835
351,351,urumqi,43.8010,87.6005,20.53,35,0,2.00,CN,1684557867
382,382,deschutes river woods,43.9915,-121.3584,18.01,66,0,2.57,US,1684557873
413,413,vila velha,-20.3297,-40.2925,19.41,84,0,2.57,BR,1684557878
479,479,woolgoolga,-30.1106,153.2007,20.58,41,0,0.96,AU,1684557887
496,496,smithers,54.7804,-127.1743,19.18,39,0,3.09,CA,1684557639
519,519,bani suwayf,29.0744,31.0979,19.82,56,0,3.83,EG,1684557892


### Step 3: Create a new DataFrame called `hotel_df`.

In [61]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_filtered[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = "" 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,namibe,AO,-15.1961,12.1522,70,
86,margaret river,AU,-33.9500,115.0667,32,
180,bom jesus da lapa,BR,-13.2550,-43.4181,61,
190,urangan,AU,-25.3000,152.9000,47,
351,urumqi,CN,43.8010,87.6005,35,
382,deschutes river woods,US,43.9915,-121.3584,66,
413,vila velha,BR,-20.3297,-40.2925,84,
479,woolgoolga,AU,-30.1106,153.2007,41,
496,smithers,CA,54.7804,-127.1743,39,
519,bani suwayf,EG,29.0744,31.0979,56,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
            "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
namibe - nearest hotel: Hotel Chik Chik Namibe
margaret river - nearest hotel: Margaret River Hotel
bom jesus da lapa - nearest hotel: No hotel found
urangan - nearest hotel: Ramada
urumqi - nearest hotel: 百时快捷酒店红山店
deschutes river woods - nearest hotel: WorldMark Bend - Seventh Mountain Resort
vila velha - nearest hotel: Hotel Prainha
woolgoolga - nearest hotel: Darlington Beach Resort
smithers - nearest hotel: Sunshine Inn Hotel
bani suwayf - nearest hotel: قاعة المؤتمرات


,City,Country,Lat,Lng,Humidity,Hotel Name
60,namibe,AO,-15.1961,12.1522,70,Hotel Chik Chik Namibe
86,margaret river,AU,-33.9500,115.0667,32,Margaret River Hotel
180,bom jesus da lapa,BR,-13.2550,-43.4181,61,No hotel found
190,urangan,AU,-25.3000,152.9000,47,Ramada
351,urumqi,CN,43.8010,87.6005,35,百时快捷酒店红山店
382,deschutes river woods,US,43.9915,-121.3584,66,WorldMark Bend - Seventh Mountain Resort
413,vila velha,BR,-20.3297,-40.2925,84,Hotel Prainha
479,woolgoolga,AU,-30.1106,153.2007,41,Darlington Beach Resort
496,smithers,CA,54.7804,-127.1743,39,Sunshine Inn Hotel
519,bani suwayf,EG,29.0744,31.0979,56,قاعة المؤتمرات


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)